# Exploration MERRA2 simulation from GMAO
------------------------------------------------

- author: Sylvie Dagoret-Campagne
- creation Friday 25 Novembre 2016
- checked : 0K January 6th 2017 with pyearth conda environnment


### purpose:

Analyse MERRA2 run for LSST air transparency simulation


http://disc.sci.gsfc.nasa.gov/datareleases/merra_2_data_release


In [ ]:
# Set up matplotlib and use a nicer set of plot parameters
%config InlineBackend.rc = {}
import matplotlib
import matplotlib as mpl
matplotlib.rc_file("templates/matplotlibrc")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
import re
import numpy as np
from mpl_toolkits.basemap import Basemap
from matplotlib import colors
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
from datetime import datetime, timedelta

In [ ]:
import h5py

In [ ]:
import libGMAOMERRA2Data as merra2  # My own library

In [ ]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

In [ ]:
############################################################################
def ensure_dir(f):
    d = os.path.dirname(f)
    if not os.path.exists(f):
        os.makedirs(f)
#########################################################################

## 1) Access to the file through pyhdf
-------------------------

### 1.1 Setting the path to the data
--------------------------------

In [ ]:
#os.environ["HDFEOS_ZOO_DIR"] = "/Users/dagoret-campagnesylvie/MacOSX/LSST/MyWork/GitHub/GMAOMERRA2data/inst1_2d_asm_Nx_M2I1NXASM"
os.environ["HDFEOS_ZOO_DIR"] = "/sps/lsst/data/AtmosphericCalibration/MERRA-2/May-Jun-2017/subset_M2I1NXASM_V5.12.4_20180424_201411"

In [ ]:
#HDFEOS_ZOO_DIR="/Users/dagoret-campagnesylvie/MacOSX/LSST/MyWork/GitHub/GMAOMERRA2data/inst1_2d_asm_Nx_M2I1NXASM"
HDFEOS_ZOO_DIR="/sps/lsst/data/AtmosphericCalibration/MERRA-2/May-Jun-2017/subset_M2I1NXASM_V5.12.4_20180424_201411"

In [ ]:
path=HDFEOS_ZOO_DIR

In [ ]:
DATAFIELD_NAME =  'TO3'

In [ ]:
DATAFIELD_UNIT =  'Db'

### 1.2 Getting the list of the files
------------------------------

In [ ]:
nc4_files = [f for f in os.listdir(path) if f.endswith('.nc4')]  
print nc4_files
full_nc4file=nc4_files
ix=0
for file in nc4_files:
    fname = os.path.join(path, file)
    full_nc4file[ix]=fname
    ix=ix+1    
print full_nc4file[0]

### 1.3 Select a given file and open it

In [ ]:
fileindex=0 # selecting the file to open : August 2016 

In [ ]:
FILE_NAME=full_nc4file[fileindex] 

In [ ]:
nc4f= h5py.File(FILE_NAME, mode='r')

In [ ]:
nc4f.items()

In [ ]:
#nc4f??

## Build Top directory and figure filename

In [ ]:
base_filename=os.path.basename(FILE_NAME).split('.nc4')[0]

In [ ]:
print base_filename

In [ ]:
p = re.compile('[.]')
root_filename=p.sub('_',base_filename)    
rootimg_dir=os.path.join('images',root_filename)

In [ ]:
rootimg_dir

In [ ]:
ensure_dir(rootimg_dir)

In [ ]:
pegfile = "{0}_{1}.jpg".format(root_filename, DATAFIELD_NAME)

In [ ]:
figfilename=root_filename+'_'+ DATAFIELD_NAME+".pdf"
print figfilename

In [ ]:
fullfigfilename=os.path.join(rootimg_dir,figfilename)

In [ ]:
fullfigfilename

### 1.4 retrieve the 3D Data

In [ ]:
name = '/TO3'
data = nc4f[name][0,:,:]
units = nc4f[name].attrs['units']
long_name = nc4f[name].attrs['long_name']
_FillValue = nc4f[name].attrs['_FillValue']
data[data == _FillValue] = np.nan
data = np.ma.masked_where(np.isnan(data), data)    

In [ ]:
time_long_name = nc4f['/time'].attrs['long_name']  

In [ ]:
# Get the geolocation data.
latitude = nc4f['/lat'][:]
longitude = nc4f['/lon'][:]

# Get the time data.
time = nc4f['/time'][:]
time_units = nc4f['/time'].attrs['units']

In [ ]:
time

In [ ]:
print time_units

In [ ]:
start_time = re.findall("^minutes since[ ]([0-9.].+[0-9.].+[0-9.].+)[ ]00:00:00$",time_units)

In [ ]:
print start_time

In [ ]:
time_rng = pd.date_range(start_time[0], periods=time.shape[0], freq='H')

In [ ]:
time_rng

In [ ]:
ts = pd.Series(np.random.randn(len(time_rng)), index=time_rng)

In [ ]:
plt.figure(figsize=(15,4))
ts.plot()

In [ ]:
pd.to_datetime(ts, format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
pp = PdfPages(fullfigfilename)

### 1.4 Data analysis
----------------------

#### 1.4.1 list of quantities to analyse

### Location of the CTIO site
------------------------------------
- Longitude = -70.815 deg
- Latitude  = -30.165277777777778 deg
- Quantity  = 2214.9999999993697 m

In [ ]:
#CTIO site
Longitude_ctio = -70.815  # deg
Latitude_ctio = -30.165277777777778 #deg
Altitude_ctio = 2214.9999999993697 #m

## 2 Analysis of Ozone
--------------------------------------

In [ ]:
fig=plt.figure(figsize=(14,6))
#X,Y=np.meshgrid(all_longitude,all_latitude)
im = plt.pcolormesh(longitude,latitude,data, cmap='hot')
plt.colorbar(im, orientation='vertical')
plt.axis([-180., 180., -90, 90 ])
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.plot([Longitude_ctio],[Latitude_ctio],'bo',markersize=10)
title=root_filename
plt.title(title)
plt.tight_layout()
plt.savefig(pp, format='pdf')
plt.show()

##### Using basemap

In [ ]:
X,Y=np.meshgrid(longitude,latitude)

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
plt.figure(figsize=(14,14))
map = Basemap()
map.drawcoastlines(color="white")
map.drawcountries(color="black")
img=map.contourf(X,Y, data,100)
cbar=map.colorbar(img,"right", size="5%", pad="2%")
cbar.set_label(DATAFIELD_UNIT)
map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
map.plot(xpt_ctio,ypt_ctio,marker='o',color='r',markersize=8)  # plot a red dot there
title=root_filename+'_'+ DATAFIELD_NAME
plt.title(title)
#plt.tight_layout() 
plt.savefig(pp, format='pdf')
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
plt.figure(figsize=(14,6))
map = Basemap()
map.drawcoastlines(color="blue")
map.drawcountries(color="black")
#img=map.contourf(X, Y, aod_clean,100)
img = plt.pcolormesh(longitude,latitude,data)
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(-40,61.,15.),labels=[True,False,False,False])
map.drawmeridians(np.arange(-180.,180.,15.),labels=[True,False,False,True])
xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
map.plot(xpt_ctio,ypt_ctio,'ro',markersize=8)  # plot a red dot there
#plt.text(xpt_lsst-100000,ypt_lsst-200000,'LSST (%5.1fW,%3.1fN)' % (Longitude_lsst,Latitude_lsst),color='red', fontsize=15)
title=root_filename
plt.title(title)
plt.tight_layout() 
plt.savefig(pp, format='pdf')
plt.show()

#### South America
-----------------

In [ ]:
#LSST site
Longitude_lsst = -70.7366833333333 # deg
Latitude_lsst = -30.240741666666672 #deg
Altitude_lsst = 2749.999999999238 #m

#CTIO Site
Longitude_ctio = -70.815 # deg
Latitude_ctio = -30.165277777777778 #deg
Altitude_ctio = 2214.9999999993697 #m

# Cerro Paranal
Longitude_paranal = -70.40300000000002 #deg
Latitude_paranal  = -24.625199999999996 #deg
Altitude_paranal = 2635.0000000009704 #m

# Observatoire de Haute Provence
Longitude_ohp=5.71222222222
Latitude_ohp=43.9316666667
Altitude_ohp=650.

#### define south america

- longitude -100° to -30°
- latitude -55° to 15°

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
#map = Basemap(llcrnrlon=-100,llcrnrlat=-60,urcrnrlon=-25.,urcrnrlat=15.,resolution='i', projection='tmerc', lat_0 = -30, lon_0 = -70)

map = Basemap(llcrnrlon=-100,llcrnrlat=-55,urcrnrlon=-30.,urcrnrlat=15., projection='tmerc', lat_0 = -30, lon_0 = -70)

map.drawmapboundary(fill_color='aqua')
map.fillcontinents(color='peachpuff',lake_color='lightskyblue')
map.drawcoastlines()
map.drawcountries()
map.drawparallels(np.arange(-50,10,5.),labels=[True,True,True,True])
map.drawmeridians(np.arange(-90.,-40.,5.),labels=[True,True,True,True])

xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
# convert back to lat/lon
#lonpt, latpt = map(xpt_lsst,ypt_lsst,inverse=True)
map.plot(xpt_ctio,ypt_ctio,'ro')  # plot a red dot there
# put some text next to the dot, offset a little bit
# (the offset is in map projection coordinates)
plt.text(xpt_ctio-100000,ypt_ctio-200000,'CTIO (%5.1fW,%3.1fN)' % (Longitude_ctio,Latitude_ctio),color='red', fontsize=15)
plt.tight_layout()
plt.savefig(pp, format='pdf')
plt.show()


### Region and Data selection for South America

In [ ]:
LongMin=-100
LongMax=-30
LatMin=-55
LatMax=15

In [ ]:
X.shape

In [ ]:
plt.pcolormesh(X, cmap='hot')
plt.colorbar(im, orientation='vertical')

In [ ]:
plt.pcolormesh(Y, cmap='hot')
plt.colorbar(im, orientation='vertical')

In [ ]:
longitude=X
latitude=Y

In [ ]:
flags_long=np.logical_and(X>=LongMin, X<=LongMax)   # flags in X where are the selected longitudes
flags_lat=np.logical_and(Y>=LatMin, Y<=LatMax)      # flags in Y where are the selected longitudes
flags_longlat=np.logical_and(flags_long,flags_lat)  # flags where the region is selected in the long-lat matrix

(selected_lat_indexes,selected_long_indexes)=np.where(flags_longlat==True) # list of indexes


selected_long=longitude[:,selected_long_indexes] # all selected longitudes
selected_lat=latitude[selected_lat_indexes,:]    # all selected latitudes

min_long_index=np.min(selected_long_indexes)
max_long_index=np.max(selected_long_indexes)

min_lat_index=np.min(selected_lat_indexes)
max_lat_index=np.max(selected_lat_indexes)

extracted_data=data[min_lat_index:max_lat_index,min_long_index:max_long_index] # extract the data

Xsel=X[min_lat_index:max_lat_index,min_long_index:max_long_index] # extract the Long
Ysel=Y[min_lat_index:max_lat_index,min_long_index:max_long_index] # extract the lat

In [ ]:
im=plt.pcolormesh(X,Y,flags_longlat, cmap='gray')
plt.colorbar(im, orientation='vertical')

In [ ]:
(index_lat,index_long)=np.where(flags_longlat==True)

In [ ]:
print selected_long

In [ ]:
print np.where(flags_longlat==True)
selected_long_indexes=np.where(flags_longlat==True)[0]
selected_lat_indexes=np.where(flags_longlat==True)[1]

In [ ]:
selected_long_indexes

In [ ]:
selected_lat_indexes

In [ ]:
im=plt.pcolormesh(Xsel,Ysel,extracted_data, cmap='hot')
plt.colorbar(im, orientation='vertical')

In [ ]:
# check what has been done
print('selected_long_indexes=',selected_long_indexes)
print('selected_lat_indexes=',selected_lat_indexes)
print('selected_long=',selected_long)
print('selected_lat=',selected_lat)
print("flags_longlat.shape=",flags_longlat.shape)
print("flags_long.shape=",flags_long.shape)
print("flags_lat.shape=",flags_lat.shape)
print("total number of ll bins",flags_longlat.shape[0]*flags_longlat.shape[1])
print("min_long_index=",min_long_index)
print("max_long_index=",max_long_index)
print("min_lat_index=",min_lat_index)
print("max_lat_index=",max_lat_index)
print("extracted_data.shape=",extracted_data.shape)
print("extracted_data.size=",extracted_data.shape[0]*extracted_data.shape[1])
#print("extracted_aod=",extracted_aod)

In [ ]:
plt.figure(figsize=(10,8))
#newdata=np.where(flags_longlat,data  , 0 )  # clean the aod from creazy data
#image = plt.pcolormesh(X,Y,newdata, cmap='hot')
image = plt.pcolormesh(Xsel,Ysel,extracted_data)
plt.xlim(LongMin,LongMax)
plt.ylim(LatMin,LatMax)
plt.plot([Longitude_ctio],[Latitude_ctio],'ro',markersize=10)
plt.xlabel('longitude')
plt.ylabel('latitude')
title=root_filename
plt.title(title)
plt.tight_layout()
plt.colorbar(image, orientation='vertical')
plt.savefig(pp, format='pdf')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
#Xsel,Ysel=np.meshgrid(selected_long,selected_lat)
im = plt.pcolormesh(Xsel,Ysel,extracted_data, cmap='hot')
plt.colorbar(im, orientation='vertical')
plt.axis([Xsel.min(), Xsel.max(), Ysel.min(), Ysel.max()])
plt.xlabel('longitude')
plt.ylabel('latitude')
plt.plot([Longitude_ctio],[Latitude_ctio],marker='o',color='b',markersize=8)
#title="Aerosol Optical Depth AOD above europe"
title=root_filename
plt.title(title)
plt.tight_layout()
plt.savefig(pp, format='pdf')
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
map = Basemap(llcrnrlon=LongMin,llcrnrlat=LatMin,urcrnrlon=LongMax,urcrnrlat=LatMax, projection='tmerc', lat_0 = Latitude_ctio, lon_0 = Longitude_ctio)
#map.drawmapboundary(fill_color='aqua') # No fill color
#map.fillcontinents(color='peachpuff',lake_color='lightskyblue') # No fill color
xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
# convert back to lat/lon
#lonpt, latpt = map(xpt_lsst,ypt_lsst,inverse=True)
map.plot(xpt_ctio,ypt_ctio,'ro')  # plot a red dot there

img=map.pcolormesh(Xsel,Ysel,extracted_data,shading='flat',latlon=True)
#img=map.contourf(Xsel, Ysel, extracted_aod,100)
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(LatMin,LatMax,5.),labels=[True,False,False,False])
map.drawmeridians(np.arange(LongMin,LongMax,5.),labels=[True,True,False,True])
map.drawcoastlines(color='yellow')
map.drawcountries(color='white')
xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
map.plot(xpt_ctio,ypt_ctio,'ro',markersize=10)  # plot a red dot there
plt.text(xpt_ctio-1000,ypt_ctio-1000,'CTIO (%5.1fW,%3.1fN)' % (Longitude_ctio,Latitude_ctio),color='yellow', fontsize=15)
title="O3 column depth South America " + root_filename
plt.title(title)
plt.tight_layout()
plt.savefig(pp, format='pdf')
plt.show()

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
map = Basemap(llcrnrlon=LongMin,llcrnrlat=LatMin,urcrnrlon=LongMax,urcrnrlat=LatMax, projection='tmerc', lat_0 = Latitude_ctio, lon_0 = Longitude_ctio)
xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
# convert back to lat/lon
#lonpt, latpt = map(xpt_lsst,ypt_lsst,inverse=True)
map.plot(xpt_ctio,ypt_ctio,'ro')  # plot a red dot there
img=map.pcolormesh(Xsel,Ysel,extracted_data,shading='flat',cmap="hot",latlon=True)
#img=map.contourf(Xsel, Ysel, extracted_aod,20)
map.colorbar(img,"right", size="5%", pad="2%")
map.drawparallels(np.arange(LatMin,LatMax,5.),labels=[True,False,False,False])
map.drawmeridians(np.arange(LongMin,LongMax,5.),labels=[True,True,False,True])
map.drawcoastlines(color='yellow')
map.drawcountries(color='white')
xpt_ctio,ypt_ctio = map(Longitude_ctio,Latitude_ctio)
map.plot(xpt_ctio,ypt_ctio,'bo',markersize=10)  # plot a red dot there
plt.text(xpt_ctio-1000,ypt_ctio-1000,'CTIO (%5.1fW,%3.1fN)' % (Longitude_ctio,Latitude_ctio),color='green', fontsize=15)
title="O3 column depth South America "+root_filename
plt.title(title)
plt.tight_layout()
plt.savefig(pp, format='pdf')
plt.show()

#### Find O3 data for CTIO site
------------------------------

In [ ]:
(NbBinLat,NbBinLong)=longitude.shape

In [ ]:
DeltaBinLat=180./float(NbBinLat)
DeltaBinLong=360./float(NbBinLong)

In [ ]:
print "dlat = ",DeltaBinLat
print "dlong = ",DeltaBinLong

In [ ]:
ctio_flags_long=np.logical_and(X>=Longitude_ctio-DeltaBinLong/2., X<=Longitude_ctio+DeltaBinLong/2.)   # flags in X where are the selected longitudes
ctio_flags_lat=np.logical_and(Y>=Latitude_ctio-DeltaBinLat/2., Y<=Latitude_ctio+DeltaBinLat/2.)      # flags in Y where are the selected longitudes
ctio_flags_longlat=np.logical_and(ctio_flags_long,ctio_flags_lat)  # flags where the region is selected in the long-lat matrix

(ctio_selected_lat_indexes,ctio_selected_long_indexes)=np.where(ctio_flags_longlat==True) # list of indexes


ctio_selected_long=longitude[:,ctio_selected_long_indexes] # all selected longitudes
ctio_selected_lat=latitude[ctio_selected_lat_indexes,:] 

ctio_min_long_index=np.min(ctio_selected_long_indexes)
ctio_max_long_index=np.max(ctio_selected_long_indexes)

ctio_min_lat_index=np.min(ctio_selected_lat_indexes)
ctio_max_lat_index=np.max(ctio_selected_lat_indexes)

ctio_extracted_data=data[ctio_min_lat_index:ctio_max_lat_index+1,ctio_min_long_index:ctio_max_long_index+1] # extract the data

In [ ]:
print('ctio_selected_long_indexes=',ctio_selected_long_indexes)
print('ctio_selected_lat_indexes=',ctio_selected_lat_indexes)
print('ctio_min_lat_index=',ctio_min_lat_index)
print('ctio_max_lat_index=',ctio_max_lat_index)
print('ctio_min_long_index=',ctio_min_long_index)
print('ctio_max_long_index=',ctio_max_long_index)
#print('ctio_selected_lat=',ctio_selected_lat)

print('A) ctio_data = ',ctio_extracted_data[0][0],' Db Unit')
print('B) ctio_data = ',data[ctio_min_lat_index,ctio_min_long_index],' Db Unit')

In [ ]:
data.shape

In [ ]:
im=plt.pcolormesh(X,Y,ctio_flags_longlat, cmap='gray')
plt.colorbar(im, orientation='vertical')

### Test library libGMAOMERRA2Data

In [ ]:
fullfigfilename

In [ ]:
(data3D,unit,longname)=merra2.GetGeoRefData(FILE_NAME,DATAFIELD_NAME)
m_data= data3D[:,:,:] ## Ozone has no additional dimensions
(m_lat,m_un_lat,m_nm_lat) = merra2.Get1DData(FILE_NAME,'lat')
m_latitude = m_lat[:]
(m_lon,m_un_lon,m_nm_lon) = merra2.Get1DData(FILE_NAME,'lon')
m_longitude = m_lon[:]
(m_tim,m_un_tim,m_nm_tim)= merra2.Get1DData(FILE_NAME,'time')
m_time=m_tim[:]

In [ ]:
m_X,m_Y=np.meshgrid(m_longitude,m_latitude)

In [ ]:
loc=merra2.observatory_location('ctio')

In [ ]:
(sel_long,sel_lat)=merra2.GetBinIndex(m_X,m_Y,loc[0],loc[1],DLong=DeltaBinLong,DLat=DeltaBinLong)

In [ ]:
m_data[:,sel_lat,sel_long]

### Find size of MERRA2 box at CTIO location

In [ ]:
(NbBinLat,NbBinLong)=longitude.shape

In [ ]:
DeltaBinLat=180./float(NbBinLat)
DeltaBinLong=360./float(NbBinLong)

In [ ]:
DeltaBinLat

In [ ]:
#distance along longitude
x1,y1 = map(Longitude_ctio-DeltaBinLong/2.,Latitude_ctio)
x2,y2 = map(Longitude_ctio+DeltaBinLong/2.,Latitude_ctio)
d_long=np.sqrt((x2-x1)**2+(y2-y1)**2)/1000

In [ ]:
print x1,x2,y1,y2
print d_long,' km'

In [ ]:
#distance along latitude
x1,y1 = map(Longitude_ctio,Latitude_ctio-DeltaBinLat/2.)
x2,y2 = map(Longitude_ctio,Latitude_ctio+DeltaBinLat/2.)
d_lat=np.sqrt((x2-x1)**2+(y2-y1)**2)/1000

In [ ]:
print x1,x2,y1,y2
print d_lat,' km'

## Close the pdf file
-------------------------

In [ ]:
pp.close()

In [ ]:
X.shape

In [ ]:
180./361

In [ ]:
360./576